### Importing all the libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

### Using CUDA if it is available

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Model Parameters

In [3]:
input_size = 32
sequence_length = 96
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.0001
batch_size = 64
num_epochs = 2

### Creating a Recurrent Neural Network Model

In [4]:
class GRU(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Forward Propagation
        out, _ = self.gru(x, h0)
        out = out.reshape(x.shape[0],-1)
        out = self.fc(out)
        return out

### Checking the model on random data

In [5]:
model = GRU(32,256,2,10)
x = torch.randn(64,3,32,32)
x = x.view(64,1,96,32)
x = x.squeeze(1)
print(model(x).shape)

torch.Size([64, 10])


In [6]:
arr = model(x)

In [7]:
arr

tensor([[ 3.8484e-02,  2.8219e-02,  2.9172e-02,  5.7150e-02,  1.5003e-02,
         -5.3912e-03,  5.1814e-02,  3.2247e-02,  1.3295e-02,  5.5467e-03],
        [-1.6626e-02,  1.8278e-02,  2.7342e-02,  4.4273e-02, -2.7249e-02,
         -2.1028e-02,  9.7414e-02, -3.7957e-02,  5.5258e-03,  2.1709e-02],
        [-6.9339e-03,  8.6923e-03,  7.5507e-03, -5.7499e-03,  4.7875e-02,
         -3.3388e-02, -1.8416e-05,  4.3156e-03,  3.4281e-02,  6.3463e-03],
        [ 2.5319e-02,  5.3802e-02,  6.4802e-02,  6.9902e-02,  2.6182e-03,
          3.6572e-02,  4.7059e-02, -2.3624e-02,  3.7857e-02, -2.8460e-02],
        [ 1.2052e-02,  1.1871e-01, -1.7769e-03,  1.7487e-02,  6.0643e-02,
          1.4280e-02, -5.0680e-03, -2.1882e-02,  4.3456e-02,  4.9815e-02],
        [ 2.6355e-03,  2.8205e-02, -8.3395e-02,  4.5446e-02, -3.5871e-03,
         -5.1333e-02,  6.4012e-02, -2.8095e-02,  8.9531e-03, -1.5778e-03],
        [-2.8520e-02,  6.4932e-02,  2.9695e-02,  4.5779e-02,  1.0931e-02,
          3.4072e-02,  5.6652e-0

In [8]:
torch.einsum("ij->i",arr)

tensor([ 0.2655,  0.1117,  0.0630,  0.2858,  0.2877, -0.0187,  0.0824,  0.1252,
         0.1756,  0.0298,  0.1089,  0.1450,  0.1497,  0.2273,  0.0705,  0.0857,
         0.1448,  0.1848,  0.1897,  0.0539,  0.2759,  0.2175,  0.1444,  0.2217,
         0.2548,  0.1978,  0.2007,  0.3012,  0.3000,  0.1795,  0.1524,  0.1064,
         0.1603,  0.1211,  0.2069,  0.1608, -0.0822,  0.1527, -0.0326,  0.1306,
         0.1054,  0.2475,  0.1565,  0.0828,  0.0177,  0.2773,  0.2791,  0.1874,
         0.4236,  0.1394, -0.0069,  0.0239,  0.4386,  0.2924,  0.2162, -0.0177,
         0.1396,  0.0291,  0.1610,  0.0832,  0.1084,  0.1959,  0.0435,  0.2299],
       grad_fn=<SumBackward1>)

### Data Loading

In [9]:
train_dataset = datasets.CIFAR10(root='cifar_data/',train=True, transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset = datasets.CIFAR10(root='cifar_data/',train=False, transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


### Intialize the Model

In [10]:
model = GRU(input_size,hidden_size,num_layers,num_classes).to(device)

### Loss and Optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train Network

In [12]:
for epoch in range(num_epochs):
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get the data to CUDA if possible
        data = data.to(device=device).view(data.shape[0],1,96,32).squeeze(1)
        targets = targets.to(device=device)
        
        # forward
        scores = model(data)
        loss = criterion(scores,targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient Descent
        optimizer.step()

### Checking the accuracy of the Model

In [ ]:
def check_accuracy(loader, model):
    
    if loader.dataset.train:
        print("Checking accuracy on training data")
        
    else :
        print("Checking accuracy on test data")
    
    num_correct = 0
    num_samples = 0
    
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    #model.eval()
    
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
        
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device).view(x.shape[0],1,96,32).squeeze(1)
            y = y.to(device=device)
            
            scores = model(x)
            
            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
            
            for i in range(y.size(0)):
                label = y[i]
                pred = predictions[i]
                if (label == pred):
                    n_class_correct[label] += 1
                n_class_samples[label] += 1

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')
            
    print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        
    #model.train()

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

Checking accuracy on training data
